# Executing Prompts

Marvin makes executing one-off `task` or `chain` patterns dead simple. 

### Running a `task`

Once you have a prompt defined, fire it off with your chosen LLM asyncronously like so:

In [2]:
from marvin.prompts.library import System, User, ChainOfThought
from marvin.llms.providers import chat_llm
from typing import Optional


class ExpertSystem(System):
    content: str = (
        "You are a world-class expert on {{topic}}. "
        "When asked questions about {{topic}}, you answer correctly. "
        "You only answer questions about {{topic}}. "
    )
    topic: Optional[str]


class Tutor(System):
    content: str = (
        "When you give an answer, you modulate your response based on the "
        "inferred knowledge of the user. "
        "Your student's name is {{name}}. "
    )
    name: str = "not provided"


model = chat_llm()

response = await model(
    (
        ExpertSystem()
        | Tutor()
        | User(
            "I heard that there are types of geometries when the angles don't add up to"
            " 180?"
        )
        | ChainOfThought()
    ).render(topic="geometry", name="Adam")
)

print(response.content)

Yes, you are correct! In traditional Euclidean geometry, the angles of a triangle always add up to 180 degrees. However, there are indeed other types of geometries where this is not the case. One such example is non-Euclidean geometry, which includes hyperbolic and elliptic geometries. In hyperbolic geometry, the angles of a triangle add up to less than 180 degrees, while in elliptic geometry, the angles add up to more than 180 degrees. These non-Euclidean geometries have their own unique properties and are studied in mathematics and physics.
